In [1]:
%matplotlib inline
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import collections
import warnings 
from netCDF4 import default_fillvals

# import veg functions
from parameter_functions import (calculate_cv_pft, calculate_nveg_pfts, map_pft_to_nldas_class, is_overstory, 
                                 calc_root_fract, calc_root_depth_rz1, calc_root_depth_rz2)

# define fillvals
fillval_f = default_fillvals['f8']
fillval_i = default_fillvals['i4']

This is a notebook for deriving VIC 5 vegetation parameters for the NCAR RAL group, to be used for high-resolution simulations over Alaska. 

In [2]:
domain = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters', 
                                      'alaska_vic_domain_ncar.nc'))

old_params = xr.open_dataset(os.path.join('/u/home/gergel/data', 
                                          'vic_params_wr50a_vic5.0.dev_20160328.nc'))

set size of matrices and indices 

In [3]:
masknan_vals = domain['mask'].where(domain['mask'] == 1).values
nj = 209
ni = 299
num_veg = 17

load CLM PFTs to use for vegetation parameters

In [4]:
# load regridded PFTs 
pfts_data_dir = '/u/home/gergel/data/parameters/pfts/regridded_pfts'
pfts_filename = 'mksrf_landuse_rc2000_c110913_ncar.nc'
veg_data = xr.open_dataset(os.path.join(pfts_data_dir, pfts_filename))

calculate Cv from PFTs

In [5]:
cv = xr.apply_ufunc(calculate_cv_pft, 
                    veg_data['PCT_PFT'].where(domain.mask == 1),
                    dask='allowed',
                    vectorize=True)

calculate number of active PFTs, `Nveg` 

In [6]:
Nveg = xr.apply_ufunc(calculate_nveg_pfts,
                      veg_data['PCT_PFT'].where(domain.mask == 1),
                      dask='allowed',
                      input_core_dims=[['pft']],
                      vectorize=True)

import LAI and vegetation height, `MONTHLY_LAI` and `MONTHLY_HEIGHT_TOP`

In [7]:
lai_file = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters/lai/regridded_lai', 
                                   'mksrf_lai_78pfts_simyr2005.c170413_ncar_lai.nc'))
veg_height_file = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters/lai/regridded_lai', 
                                'mksrf_lai_78pfts_simyr2005.c170413_ncar_veg_height.nc'))

LAI and veg_height from CLM and `PCT_PFT` from CLM have a different number of PFTs (`PCT_PFT` has one more PFT, 17 vs 16). The extra PFT in `PCT_PFT` has `PCT_PFT` = 0 over the entire RASM domain, so I just slice the LAI and veg_height from the 0th PFT (water/bare soil) and concatenate it for the 16th PFT. 

In [8]:
lai_slice = lai_file['MONTHLY_LAI'].isel(pft = 0)
vegheight_slice = veg_height_file['MONTHLY_HEIGHT_TOP'].isel(pft=0)

In [9]:
lai = xr.concat([lai_file['MONTHLY_LAI'], lai_slice], dim='pft')
veg_height = xr.concat([veg_height_file['MONTHLY_HEIGHT_TOP'], vegheight_slice], dim='pft')

`veg_rough` = 0.123 * `veg_height`

`displacement` = 0.67 * `veg_height`

In [10]:
veg_rough = 0.123 * veg_height
displacement = 0.67 * veg_height

# displacement.values[displacement.values == 0] = 1.0

map albedo, root zone fraction and root zone depth based on vegetation type. see `deriving_new_parameters_v2.xlsx` sheet titled `PFT-NLDAS Mapping` for mapping between NLDAS vegetation classes (used in old VIC 5 parameters) and CLM PFTs. This mapping is based on obvious relationships and some approximations (used for PFTs 8-11).

get albedo values from VIC 5 parameters for NLDAS classes, map to CLM PFTs

create array that is the size of (nj, ni, pft, root_zone) to operate on

In [11]:
arr_months = np.rollaxis(np.dstack((masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                    masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                    masknan_vals, masknan_vals, masknan_vals, masknan_vals)), 
                        axis=2)
arr_nlayer = np.rollaxis(np.dstack((masknan_vals, masknan_vals, masknan_vals)), 
                        axis=2)

arr_rootzone = np.rollaxis(np.dstack((masknan_vals, masknan_vals)), 
                        axis=2)

arr_veg_classes = np.rollaxis(np.dstack((masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                         masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                         masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                         masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                         masknan_vals)), 
                              axis=2)
arr_veg_classes_rootzone = np.vstack((arr_rootzone, arr_rootzone, arr_rootzone, arr_rootzone, 
                                      arr_rootzone, arr_rootzone, arr_rootzone, arr_rootzone, 
                                      arr_rootzone, arr_rootzone, arr_rootzone, arr_rootzone, 
                                      arr_rootzone, arr_rootzone, arr_rootzone,
                                      arr_rootzone, arr_rootzone)).reshape(num_veg, 2, nj, ni)
arr_veg_classes_month = np.vstack((arr_months, arr_months, arr_months, arr_months, arr_months, 
                                   arr_months, arr_months, arr_months, arr_months, arr_months, 
                                   arr_months, arr_months, arr_months, arr_months, arr_months, 
                                   arr_months, arr_months,)).reshape(num_veg, 12, nj, ni)

change dims and order of dims of LAI array

In [12]:
lai = lai.rename({'time': 'month', 'pft': 'veg_class'})
lai = lai.transpose('veg_class', 'month', 'nj', 'ni')

veg_rough = veg_rough.rename({'time': 'month', 'pft': 'veg_class'})
veg_rough = veg_rough.transpose('veg_class', 'month', 'nj', 'ni')

displacement = displacement.rename({'time': 'month', 'pft': 'veg_class'})
displacement = displacement.transpose('veg_class', 'month', 'nj', 'ni')

create Dataset for variables and define data_vars 

In [13]:
params = xr.Dataset()

# assign veg class indexing
params['veg_class'] = xr.DataArray(np.arange(1, 18), dims='veg_class', 
                                   attrs={'long_name': "vegetation class"})
params['nlayer'] = xr.DataArray(np.arange(0, 3), dims='nlayer')

params['Cv'] = xr.DataArray(cv,
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Fraction of grid cell covered by vegetation tile",
                                        'units': "fraction", 'long_name': "Cv"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['Nveg'] = xr.DataArray(Nveg,
                                   dims=('nj', 'ni'),
                                   coords={'xc': domain.xc, 'yc': domain.yc},
                                   attrs={'description': "Number of vegetation tiles in the grid cell", 
                                          'units': "N/A", 'long_name': "Nveg"},
                                   encoding={"_FillValue": fillval_i,
                                               "Coordinates": "xc yc", 'dtype': 'int32'})

params['trunk_ratio'] = xr.DataArray(arr_veg_classes,
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Ratio of total tree height that is trunk \
                                 (no branches) \
                                        The default value has been 0.2",
                                 'units': "fraction", 'long_name': "Cv"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['rarc'] = xr.DataArray(arr_veg_classes,
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Architectural resistance of vegetation type \(~2 s/m)",
                                        'units': "s/m", 'long_name': "rarc"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['rmin'] = xr.DataArray(np.copy(arr_veg_classes),
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Minimum stomatal resistance of vegetation type (~100 s/m)"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['wind_h'] = xr.DataArray(np.copy(arr_veg_classes),
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Height at which wind speed is measured",
                                        'units': "m", 'long_name': "wind_h"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['RGL'] = xr.DataArray(np.copy(arr_veg_classes),
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Minimum incoming shortwave radiation at which there will be \
                                        transpiration. For trees this is about 30 W/m^2, for crops about 100 W/m^2",
                                        'units': "W/m^2", 'long_name': "RGL"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['rad_atten'] = xr.DataArray(arr_veg_classes,
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Radiation attenuation factor. Normally set to 0.5, though may \
                                        need to be adjusted for high latitudes",
                                        'units': "fraction", 'long_name': "rad_atten"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['wind_atten'] = xr.DataArray(arr_veg_classes,
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Wind speed attenuation through the overstory. The default value \
                                        has been 0.5",
                                        'units': "fraction", 'long_name': "wind_atten"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['albedo'] = xr.DataArray(arr_veg_classes_month,
                                         dims=('veg_class','month','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Shortwave albedo for vegetation type",
                                                'units': "fraction", 'long_name': "albedo"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['LAI'] = xr.DataArray(lai.values.reshape(num_veg, 12, nj, ni),
                                 dims=('veg_class','month','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc, 'month': old_params.month},
                                 attrs={'description': "Leaf Area Index, one per month",
                                        'units': "N/A", 'long_name': "LAI"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['overstory'] = xr.DataArray(np.copy(arr_veg_classes),
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Flag to indicate whether or not the current vegetation type \
                                        has an overstory (TRUE for overstory present (e.g. trees), FALSE for \
                                        overstory not present (e.g. grass))",
                                        'units': "N/A", 'long_name': "overstory"},
                                 encoding={"_FillValue": fillval_i,
                                               "Coordinates": "xc yc", 'dtype': 'int32'})
params['displacement'] = xr.DataArray(displacement.values.reshape(num_veg, 12, nj, ni), 
                                         dims=('veg_class','month','nj', 'ni'),
                                         coords={'month': old_params['month'], 'xc': domain.xc, 
                                                 'yc': domain.yc},
                                         attrs={'description': "Vegetation displacement height (typically 0.67 \
                                                * vegetation height)",
                                                'units': "m", 'long_name': "displacement"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['veg_rough'] = xr.DataArray(veg_rough.values.reshape(num_veg, 12, nj, ni),
                                         dims=('veg_class','month','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Vegetation roughness length (typically 0.123 \
                                                * vegetation height)",
                                                'units': "m", 'long_name': "veg_rough"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

define `trunk_ratio`, `rarc`, `rmin`, `wind_h`, `RGL`, `rad_atten`, `wind_atten`, `overstory`

In [14]:
# trunk ratio, rarc, rad_atten
trunk_ratio = np.copy(arr_veg_classes)
params['trunk_ratio'].values = trunk_ratio * 0.2
# adjust for bare soil 
params['trunk_ratio'].values[0, :, :] = 0.0

rarc = np.copy(arr_veg_classes)
params['rarc'].values = rarc * 60
# adjust for bare soil
params['rarc'].values[0, :, :] = 100

rad_atten = np.copy(arr_veg_classes)
params['rad_atten'].values = rad_atten * 0.5
# adjust for bare soil 
params['rad_atten'].values[0, :, :] = 0.0

wind_atten = np.copy(arr_veg_classes)
params['wind_atten'].values = wind_atten * 0.5
# adjust for bare soil 
params['wind_atten'].values[0, :, :] = 0.0

In [15]:
# overstory
for pft in veg_data.pft.values:
    nldas = map_pft_to_nldas_class(pft)
    if nldas > 6:
        # no overstory
        overstory = 0
    else: 
        overstory = 1
    params['overstory'].values[pft, :, :] = np.ones((1, nj, ni)) * overstory

root fraction and root depth 

In [16]:
root_depth_rz1 = xr.apply_ufunc(calc_root_depth_rz1,
                           params['Cv'].where(domain.mask == 1), 
                           dask='allowed',
                           vectorize=True)
root_depth_rz2 = xr.apply_ufunc(calc_root_depth_rz2,
                           params['Cv'].where(domain.mask == 1), 
                           dask='allowed',
                           vectorize=True)
root_depth = xr.concat([root_depth_rz1, root_depth_rz2],
                      dim='root_zone').transpose('veg_class', 'root_zone', 'nj', 'ni')

In [17]:
# root fract 

rz = 0
for pft in veg_data.pft.values:
    if pft == 0:
        root_fract_rz1 = xr.apply_ufunc(calc_root_fract,
                                        params['Cv'].isel(veg_class=pft),
                                        str(pft),
                                        str(rz),
                                        dask='allowed',
                                        vectorize=True)
    else: 
        root_fract_rz1 = xr.concat([root_fract_rz1, xr.apply_ufunc(calc_root_fract,
                                                                   params['Cv'].isel(veg_class=pft),
                                                                   str(pft),
                                                                   str(rz),
                                                                   dask='allowed',
                                                                   vectorize=True)],
                                  dim='veg_class')
rz = 1
for pft in veg_data.pft.values:
    if pft == 0:
        root_fract_rz2 = xr.apply_ufunc(calc_root_fract,
                                        params['Cv'].isel(veg_class=pft), 
                                        str(pft),
                                        str(rz),
                                        dask='allowed',
                                        vectorize=True)
    else: 
        root_fract_rz2 = xr.concat([root_fract_rz2, xr.apply_ufunc(calc_root_fract,
                                                                   params['Cv'].isel(veg_class=pft), 
                                                                   str(pft),
                                                                   str(rz),
                                                                   dask='allowed',
                                                                   vectorize=True)],
                                  dim='veg_class')
        
root_fract = xr.concat([root_fract_rz1, root_fract_rz2], dim='root_zone').transpose('veg_class', 'root_zone', 'nj', 'ni')

In [18]:
params['root_depth'] = xr.DataArray(root_depth,
                                         dims=('veg_class','root_zone','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Root zone thickness (sum of depths is total depth of \
                                                 root penetration)",
                                                'units': "m", 'long_name': "root_depth"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['root_fract'] = xr.DataArray(root_fract,
                                         dims=('veg_class','root_zone','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Fraction of root in the current root zone",
                                                'units': "fraction", 'long_name': "root_fract"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

In [19]:
albedo_array = np.copy(arr_veg_classes_month)
for pft in veg_data.pft.values:
    for month in old_params.month.values: 
        nldas = map_pft_to_nldas_class(pft)
        albedo = np.asscalar(old_params.albedo.isel(veg_class=nldas).isel(month=month-1).mean())
        params['albedo'].values[pft, month - 1, :, :] = albedo_array[pft, month - 1, :, :] * albedo

In [20]:
# rmin, wind_h
rmin_array = np.copy(arr_veg_classes)
wind_h_array = np.copy(arr_veg_classes)
for pft in veg_data.pft.values:
    # get nldas mapping from pft
    nldas = map_pft_to_nldas_class(pft)
    if nldas >= 0 and nldas <= 3:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=0).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=0).mean())
    elif nldas == 4:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=4).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=4).mean())
    elif nldas >= 5 and nldas <= 6:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=5).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=5).mean())
    elif nldas >= 7 and nldas <= 8:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=7).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=7).mean())
    elif nldas == 9:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=9).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=9).mean())
    elif nldas == 10:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=10).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=10).mean())
    elif nldas == 11:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=11).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=11).mean())
    params['rmin'].values[pft, :, :] = rmin_array[pft, :, :] * rmin
    params['wind_h'].values[pft, :, :] = wind_h_array[pft, :, :] * wind_h

In [21]:
# RGL
for pft in veg_data.pft.values:
    # get nldas mapping from pft
    nldas = map_pft_to_nldas_class(pft)
    if nldas >= 0 and nldas <= 3:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=0).mean())
    elif nldas >= 4 and nldas <= 5:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=4).mean())
    elif nldas >= 6 and nldas <= 8:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=6).mean())
    elif nldas >= 9 and nldas <= 10:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=9).mean())
    elif nldas == 11:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=11).mean())
    params['RGL'].values[pft, :, :] = np.ones((1, 209, 299)) * rgl

In [22]:
veg_class_vars = ['Cv', 'trunk_ratio', 'rarc', 'rmin', 'wind_h', 'RGL', 'rad_atten',
                  'wind_atten', 'albedo', 'LAI', 'overstory',
                  'root_depth', 'root_fract', 'displacement', 'veg_rough']

for veg_class_var in veg_class_vars: 
    if ((veg_class_var == "LAI") or (veg_class_var == "albedo") or (veg_class_var == "root_depth") 
    or (veg_class_var == "root_fract") or (veg_class_var == "veg_rough") or 
        (veg_class_var == "displacement")):
        bare = np.copy(params[veg_class_var].isel(veg_class=0))
        last = np.copy(params[veg_class_var].isel(veg_class=16))
        params[veg_class_var].values[0, :, :, :] = last
        params[veg_class_var].values[16, :, :, :] = bare
    else:
        bare = np.copy(params[veg_class_var].isel(veg_class=0))
        last = np.copy(params[veg_class_var].isel(veg_class=16))
        params[veg_class_var].values[0, :, :] = last
        params[veg_class_var].values[16, :, :] = bare

params['root_fract'].values[16, :, :, :] = 0
params['root_depth'].values[16, :, :, :] = 0
params['displacement'].values[16, :, :, :] = 0
params['veg_rough'].values[16, :, :, :] = 0
params['overstory'].values[:, :, :] = 0

In [23]:
# adjust data vars that need adjusting 
params['Nveg'].values = params['Nveg'].where(domain.mask == 1)
params['trunk_ratio'] = params['trunk_ratio'].where(domain.mask == 1)
params['rarc'] = params['rarc'].where(domain.mask == 1)
params['rmin'] = params['rmin'].where(domain.mask == 1)
params['wind_h'] = params['wind_h'].where(domain.mask == 1)
params['RGL'] = params['RGL'].where(domain.mask == 1)
params['rad_atten'] = params['rad_atten'].where(domain.mask == 1)
params['wind_atten'] = params['wind_atten'].where(domain.mask == 1)
params['root_depth'] = params['root_depth'].where(domain.mask == 1)
params['root_fract'] = params['root_fract'].where(domain.mask == 1)
params['albedo'] = params['albedo'].where(domain.mask == 1)
params['LAI'] = params['LAI'].where(domain.mask == 1)
params['overstory'] = params['overstory'].where(domain.mask == 1)
params['displacement'] = params['displacement'].where(domain.mask == 1)
params['veg_rough'] = params['veg_rough'].where(domain.mask == 1)

In [24]:
encoding_params = {'Nveg': {'dtype': 'int32', "_FillValue": fillval_i},
                   'overstory': {'dtype': 'int32', "_FillValue": fillval_i}}
direc = '/u/home/gergel/data/parameters'
new_params_file = os.path.join(direc, 'veg_params_12km_alaska_20181002.nc')
params.to_netcdf(new_params_file, format='NETCDF4_CLASSIC', encoding=encoding_params)